## Load a pretrained model

In [1]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [2]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

## Inference (two variants)

In [3]:
from transformers import pipeline

prompt = 'Ну привет'
generator = pipeline("text-generation", model="sberbank-ai/rugpt3small_based_on_gpt2")
res = generator(prompt, max_new_tokens=50)
print(res[0]['generated_text'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ну привет, — сказал он. — Я тут подумал, что ты, наверное, не знаешь, что я здесь.

— Я знаю, — сказала я. — Я знаю, что ты здесь.

— Я


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
inputs = tokenizer(prompt, return_tensors="pt").input_ids
inputs

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


tensor([[7448, 6129]])

In [5]:
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=5, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ну привет!\n\n—\xa0Привет!\xa0— отозвался я, с любопытством глядя на девушку.\xa0— А я вот, кажется, узнал тебя.\n\n—\xa0Да, это я,\xa0— ответила она, и я сразу понял, что это не она.\n\n—\xa0А я,\xa0— кивнул я.\xa0— Меня зовут Сергей.\n\n—\xa0А меня — Катя.\n\n—\xa0Очень приятно,\xa0— улыбнулся я.\xa0— Меня зовут Сергей.\n\n']

## Dataset Preparation

In [6]:
import pandas as pd

data = pd.read_csv('/kaggle/input/mayakovsky/Mayakovsky.txt', delimiter='|', names=['text'])
data

,text
0,КО ВСЕМУ
1,"Нет. Это неправда. Нет! И ты? Любимая, за что,..."
2,"ложек! Белый, сшатался с пятого этажа. Ветер щ..."
3,Вознес над суетой столичной одури строгое древ...
4,В грубом убийстве не пачкала рук ты. Ты уронил...
...,...
9833,поэтических
9834,"рвачей и выжиг я подыму,"
9835,"как большевистский партбилет, все сто томов"
9836,моих


In [7]:
data = data.sample(frac=1.0)
data

,text
887,Я подымаю рельсовый бунт.
1723,"как знанье,"
6513,"такие же названия, как любимым"
1241,Если в будущее
6280,а пишущий
...,...
255,"Чтоб бешеной пляской землю овить, скучную, как..."
9302,редактор. Стал вынослив
28,"Грудой дел,"
3425,от жары балда. Кто над морем не философствовал...


In [8]:
import numpy as np

np.sum(data.isnull())

text    0
dtype: int64

In [9]:
bad_lines = data.loc[data.text.str.match('\d{4}')]
print(np.unique(bad_lines.values))
print(bad_lines.size)

['1912 ' '1913 ' '1914 ' '1915 ' '1916 ' '1917 ' '1917 - 1926 ' '1918 '
 '1919 ' '1919, октябрь ' '1920 ' '1920 - 1921 ' '1920, август '
 '1920, июль ' '1921 ' '1921, январь ' '1922 ' '1922 - 1923 ' '1923 '
 '1924 ' '1925 ' '1926 ' '1926 Ялта, Симферополь, Гурзуф, Алупка ' '1927'
 '1927 ' '1929 ' '1930 ']
183


In [10]:
clean_data = data.drop(bad_lines.index)
clean_data

,text
887,Я подымаю рельсовый бунт.
1723,"как знанье,"
6513,"такие же названия, как любимым"
1241,Если в будущее
6280,а пишущий
...,...
255,"Чтоб бешеной пляской землю овить, скучную, как..."
9302,редактор. Стал вынослив
28,"Грудой дел,"
3425,от жары балда. Кто над морем не философствовал...


In [11]:
clean_data = clean_data.reset_index(drop=True)
clean_data

,text
0,Я подымаю рельсовый бунт.
1,"как знанье,"
2,"такие же названия, как любимым"
3,Если в будущее
4,а пишущий
...,...
9650,"Чтоб бешеной пляской землю овить, скучную, как..."
9651,редактор. Стал вынослив
9652,"Грудой дел,"
9653,от жары балда. Кто над морем не философствовал...


In [12]:
test_size = 955
train_size = clean_data.size - test_size
train_texts = clean_data.text.values[:train_size]
test_texts = clean_data.text.values[-test_size:]
train_texts[:5], test_texts[:5]

(array(['Я подымаю рельсовый бунт. ', 'как знанье, ',
        'такие же названия, как любимым ', 'Если в будущее ', 'а пишущий '],
       dtype=object),
 array(['Не смеют надеяться: с кольцом экватора ', 'сыплет ',
        'очень к вам привыкли. Ну, давайте, ',
        'не вспомнят ни зги. История ',
        'нету этаких... Мы найдем себе другую в разызысканной жакетке.Припомадясь '],
       dtype=object))

In [13]:
from datasets import Dataset, DatasetDict

dev_pcnt = 0.1
dev_size = int(train_size * dev_pcnt)
train_df = pd.DataFrame(
    {
        'text': train_texts[:-dev_size],
    }
)
dev_df = pd.DataFrame(
    {
        'text': train_texts[-dev_size:],
    }
)
test_df = pd.DataFrame(
    {
        'text': test_texts,
    }
)

data = DatasetDict(
    {
        'train': Dataset.from_pandas(train_df.reset_index(drop=True)),
        'dev': Dataset.from_pandas(dev_df.reset_index(drop=True)),
        'test': Dataset.from_pandas(test_df.reset_index(drop=True)),
    }
)
data

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7830
    })
    dev: Dataset({
        features: ['text'],
        num_rows: 870
    })
    test: Dataset({
        features: ['text'],
        num_rows: 955
    })
})

In [20]:
MAX_LENGTH = 128

def preprocess(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_LENGTH)
    
data_tokenized = data.map(
    lambda row: tokenizer(row['text']), batched=True, remove_columns=['text']
)
data_tokenized

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 7830
    })
    dev: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 870
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 955
    })
})

## Model Training

In [22]:
import gc

import torch
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir = 'mayak_model',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    weight_decay=0.01,
    save_steps=25000,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_tokenized['train'],
    eval_dataset=data_tokenized['dev'],
    data_collator=data_collator,
)

gc.collect()
torch.cuda.empty_cache()

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 7830
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5874
  Number of trainable parameters = 125231616
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss
1,4.802900,4.597870
2,3.823900,4.724614
3,3.375600,4.870719


***** Running Evaluation *****
  Num examples = 870
  Batch size = 4
***** Running Evaluation *****
  Num examples = 870
  Batch size = 4
***** Running Evaluation *****
  Num examples = 870
  Batch size = 4


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5874, training_loss=3.8203137884506937, metrics={'train_runtime': 353.1915, 'train_samples_per_second': 66.508, 'train_steps_per_second': 16.631, 'total_flos': 250149355776000.0, 'train_loss': 3.8203137884506937, 'epoch': 3.0})

In [23]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
eval_results

***** Running Evaluation *****
  Num examples = 870
  Batch size = 4


Perplexity: 130.41


{'eval_loss': 4.8707194328308105,
 'eval_runtime': 2.3426,
 'eval_samples_per_second': 371.388,
 'eval_steps_per_second': 93.061,
 'epoch': 3.0}

In [25]:
trainer.save_model('mayak_model')

Saving model checkpoint to mayak_model
Configuration saved in mayak_model/config.json
Configuration saved in mayak_model/generation_config.json
Model weights saved in mayak_model/pytorch_model.bin


In [29]:
model = AutoModelForCausalLM.from_pretrained("mayak_model")
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

loading configuration file mayak_model/config.json
Model config GPT2Config {
  "_name_or_path": "mayak_model",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 50264
}

loading weights file mayak_model/pytorch_model.bin
Generate config GenerationConfig {
 

In [44]:
prompt = 'Ну привет'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
for _ in range(5):
    outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=3, top_p=0.95, repetition_penalty=1.5)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ну приветствую, товарищ судья.  Ну что ж...  Я не против того чтоб в моем доме жили такие-то людишки и подобные им? Нет! Пусть живут себе на даче или под пальмой у моей любимой бабушки! И пусть едят мою любимую']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ну привет, ройсь! Я пришел к тебе. Не гневи бога ради: я не враг твой и даже рад тебя видеть... Но если ты думаешь меня прогнать из Кремля в полночь или день - иди вон отсюда с глаз долой!" И скрылся']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ну привет, росиянская рать! Не думал я... Думал ли - нет. Я ж не из тех, кто любит громить и грабить богатых или просто устал от войн. Но вот грянула война: "Иди!" И сразу же']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ну приветствую!  - Доброе утро, мистер Форд. Рад вас видеть... Ну как вам? Как вы себя чувствуете?..  И сразу же:  - Я в очень плохом состоянии!..  В комнате стало тихо и пусто; кромешная тьма']
['Ну приветствую вас, товарищи!  - Что ж...  Не очень приятно вам было с нами познакомиться. А теперь скажите: это вы были на съезде Советов?  Мы знаем кого мы выбирали вождем Советского Союза и кто был нашим кумиром в']


In [38]:
prompt = 'Ведь если звезды зажигают'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
for _ in range(5):
    outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=5, top_p=0.95)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают, значит,  их  кто-то  гонит  из мрака.  - Кто это?!  -  не в силах  удержаться,  чтоб не заорать:  - Кто-то  гонит  из мрака']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают в глазах, то значит,  кто-то  в сердце  горит.  А если  на небе  -  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной  сплошной']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают в сердце, значит,  они  - друзья.  И в жизни,  и в смерти  - мы  друзья,  и мы  в горе  - не друзья.  И в сердце  - наша сила.  И мы ']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают огни, то, может быть, оттого, что в небо, как в чашу, налита вода, то, может быть, оттого, что в небо налита вода, и, может быть, оттого, что в воду налита земля". ']
['Ведь если звезды зажигают,  то  чтоб  чтоб  в комнате  не было  скучно  тем,  кто  сидит  у окна  в своем кабинете.  А если  в окне  горит  лампочка,  то  чтоб  не  было']


In [39]:
prompt = 'Ведь если звезды зажигают'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
for _ in range(5):
    outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=5, top_p=0.95, repetition_penalty=1.5)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают в глазах, то значит  это кто-то чужой. А я не враг вам и вашим звездам: мне они ближе по духу! - Простите великодушно,- сказал поэт,- а где вы были? Я бы на вашем месте тоже...  Но']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают,  то их же надо смотреть! А мы живем в постоянном страхе. Мы знаем: от пули можно уйти и живым; но нам страшно даже выйти из этого самого смерча... В мире все так-то просто - не бывает ничего простого или']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают,  чтоб в небо лезли люди и чтобы из земли вылезал скелет - это хорошо. Я знаю одного знаменитого ученого: он сидит на корточках возле столика с книжкой. Он пишет роман о голоде-беде... А когда ему скучно']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают их в угоду чьим-то капризам, то значит ли это что я один из них? Нет!  - Вы не правы,- сказал он наконец.  И опять же: нет... Не может быть!.. Он встал со стула и вышел']
['Ведь если звезды зажигают, то их  огни - наши тени... Мы любим тебя. А они нас не любят! Неправда ли чудная вещь?!" И так далее в том же роде". В самом деле: "А что это за книга?" Ответил ей Петр']


In [43]:
prompt = 'Ведь если звезды зажигают'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
for _ in range(5):
    outputs = model.generate(inputs, max_new_tokens=50, do_sample=True, top_k=3, top_p=0.95, repetition_penalty=1.5)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают глаза, значит это не так уж страшно.  И вот они зажглись и засверкали ярче солнца...И сразу же за ними в небе вырос огромный черный дракон!  Он был очень велик: около трех метров длиной с лошадь; он тя']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают глаза, значит  в комнате что-то не так; и я вижу: это кто же там сидит на корточках? И вот из глаз у меня вытекла одна слеза. - Я знаю этого парня! Он живет с другими людьми... А']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают глаза, значит  они не для этого. Я знаю одного человека - он умер от голода и холода... Он жил в коммуне на берегу Днепра! Там же был убит голодным звоном зубовный скрежет!.. И вот сидит этот самый человек']


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ведь если звезды зажигают глаза, значит  они тоже люди. И в комнате у них есть диван и столик для книг... Вот так-то! А что ж им - человеку?! Человеку ведь не нужно ничего кроме него самого: его тело; душу он любит больше всего']
['Ведь если звезды зажигают в глазах, то это не от солнца. А кто их знает?  И вот однажды на небе вспыхнула яркая звезда и зажглась над миром... Венера!  Луна сияла долго-долго: - Я знаю эту ночь лучше других ночей']


In [42]:
prompt = 'Ну привет'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=50, do_sample=False)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))
prompt = 'Ведь если звезды зажигают'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=50, do_sample=False)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Ну приветствую вас, товарищи!  - Доброе утро, товарищ!  - Доброе утро, товарищ!  - Доброе утро, товарищ!  - Доброе утро, товарищ!  - Доброе утро, товарищ!  - Доброе утро']
['Ведь если звезды зажигают,  то,  может быть,  чтоб их не было видно?  И если  в небе  горит  яркая звезда,  то  это  не звезда,  а просто  яркая  яркая  звезда.  И если  в']
